In [ ]:
import cv2
import pickle
import os
import numpy as np
import datetime
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
# from tensorflow.keras.models import Sequential, load_model
from tensorflow.python import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, TensorBoard
from helpers import resize_to_fit  

# # 不区分大小写的字母文件夹
# LETTER_IMAGES_FOLDER = "extracted_letter_images"
# # 不区分大小写的模型保存位置
# MODEL_FILENAME = "captcha_model.hdf5"
# # 不区分大小写的转换对象保存位置
# MODEL_LABELS_FILENAME = "model_labels.dat"

# # 区分大小写的文件夹
LETTER_IMAGES_FOLDER = "extracted_letter_images_"
# # 区分大小写的模型保存位置
MODEL_FILENAME = "captcha_model_.hdf5"
# # 区分大小写的转换对象保存位置
MODEL_LABELS_FILENAME = "model_labels_.dat"

# initialize the data and labels
data = []
labels = []

# 循环输入图像
for image_file in paths.list_images(LETTER_IMAGES_FOLDER):   # 1
    # 加载图像并将其转换为灰度图像
    image = cv2.imread(image_file)                           # 2   
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)          # 3 

    # 改变图像大小
    image = resize_to_fit(image, 30, 30) # 引入的函数

    # A增加一个维度
    image = np.expand_dims(image, axis=2)

    # 根据字母所在的文件夹来获取字母的名称
    label = image_file.split(os.path.sep)[-2]                # 4
    if len(label)==2: 
        label = label[0]
   
    # 将字母图像及其标签添加到训练数据中
    data.append(image)
    #print(data)
    labels.append(label)

# 将原始像素调整为[0，1]范围（这可以改善训练）
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)    
    
# 分为训练集和测试集
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.2, random_state=0)

# 图像增强
image_gen_train = ImageDataGenerator(
    rotation_range=45, # 随机45度旋转
    height_shift_range=.15, #随机高度偏移 [-0.15,0.15)
)


# 将标签转换为独热码
lb = LabelBinarizer().fit(Y_train) # 构建一个转换对象
Y_train = lb.transform(Y_train)
Y_test = lb.transform(Y_test)

# 保存对象，供以后使用模型解码其预测的含义时使用
with open(MODEL_LABELS_FILENAME, "wb") as f:
    pickle.dump(lb, f)                                      # 5

class CNNWhu(keras.Model):
    # 搭建神经网络
    def __init__(self):
        super(CNNWhu, self).__init__()

        # 第一个带有最大池化层的卷积层
        self.c1 = Conv2D(32, (3, 3), 
                         padding="same", 
                         input_shape=(30, 30, 1), 
                         kernel_regularizer = keras.regularizers.l2(0.01)
                         ) # 6 bias_regularizer = keras.regularizers.l2(0.01)
        self.b1 = BatchNormalization() # BN层
        self.a1 = Activation('relu')
        self.p1 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2))
        self.d1 = Dropout(0.2)

        # 第二个带有最大池化层的卷积层
        self.c2 = Conv2D(64, (3, 3), 
                         padding="same", 
                         activation="relu" ,
                         kernel_regularizer = keras.regularizers.l2(0.01)
                         )                 
        self.p2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))

        # 第三个卷积层
        self.c3 = Conv2D(64, (3, 3), 
                         padding="same", 
                         activation="relu",
                         kernel_regularizer = keras.regularizers.l2(0.01))

        # 压缩层
        self.flatten = Flatten()
        
        # 500个神经元的隐藏层
        self.f1 = Dense(500, activation="relu")#  kernel_regularizer=keras.regularizers.l2(0.01)

        # 增加Dropout层来避免过拟合
        self.d2 = Dropout(0.3)

        # 输出层
        # model.add(Dense(36, activation="softmax"))
        self.f2 = Dense(62, activation="softmax")

        
    def call(self, x):
        x1 = self.c1(x)
        x1 = self.b1(x)
        x1 = self.a1(x)
        x1 = self.p1(x)
        x1 = self.d1(x)

        x2 = self.c2(x1)
        x2 = self.p2(x2)

        x3 = self.c3(x2)

        y = self.flatten(x3)
        y = self.f1(y)
        y = self.d2(y)
        y = self.f2(y)

        return y
        
model = CNNWhu()

# 构建模型
model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"]) # 7


current_time = datetime.datetime.now().strftime("%Y.%m.%d_%H-%M-%S")
logdir = os.path.join('cnn_callbacks_', current_time)
if not os.path.exists(logdir):
    os.mkdir(logdir)

# 设置断点
callbacks = [ModelCheckpoint(MODEL_FILENAME,monitor='val_loss',
                             save_weights_only=False, verbose=1, save_best_only=True, period=1)]         
""" # 网络可视化
            TensorBoard(log_dir= logdir,histogram_freq=1,
                        write_graph=True, write_grads=False, write_images=True,
                        embeddings_freq=0, embeddings_layer_names=None,
                        embeddings_metadata=None, embeddings_data=None, update_freq=500
                        )# tensorboard --logdir=train
"""

# 加载断点，训练神经网络

if os.path.exists(MODEL_FILENAME):
    model = load_model(MODEL_FILENAME)
    # 若成功加载前面保存的参数，输出下面的信息
    print("checkpoint_loaded")

history = model.fit_generator(image_gen_train.flow(X_train, Y_train, batch_size=32),
                              validation_data=(X_test, Y_test),
                              steps_per_epoch=len(X_train) / 32, epochs=40, 
                              callbacks = callbacks)# 

model.summary()

# 绘制训练 & 验证的准确率值
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.show()

# 绘制训练 & 验证的损失值
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()


model.evaluate(X_test, Y_test)

# model.save_weights('model_weights.h5')

# Save the trained model to disk
# model.save(MODEL_FILENAME)


checkpoint_loaded
  ...
    to  
  ['...']
Train for 65.875 steps, validate on 528 samples
Epoch 1/40
65/65 [============================>.] - ETA: 0s - loss: 0.3425 - accuracy: 0.9326
Epoch 00001: val_loss improved from inf to 0.25283, saving model to captcha_model_.hdf5
66/65 [==============================] - 4s 53ms/step - loss: 0.3411 - accuracy: 0.9326 - val_loss: 0.2528 - val_accuracy: 0.9735
Epoch 2/40
65/65 [============================>.] - ETA: 0s - loss: 0.3161 - accuracy: 0.9388
Epoch 00002: val_loss did not improve from 0.25283
66/65 [==============================] - 3s 48ms/step - loss: 0.3197 - accuracy: 0.9383 - val_loss: 0.3152 - val_accuracy: 0.9545
Epoch 3/40
64/65 [============================>.] - ETA: 0s - loss: 0.3502 - accuracy: 0.9150
Epoch 00003: val_loss did not improve from 0.25283
66/65 [==============================] - 3s 44ms/step - loss: 0.3492 - accuracy: 0.9151 - val_loss: 0.3026 - val_accuracy: 0.9451
Epoch 4/40
64/65 [============================>

Epoch 30/40
65/65 [============================>.] - ETA: 0s - loss: 0.3200 - accuracy: 0.9253
Epoch 00030: val_loss did not improve from 0.21570
66/65 [==============================] - 4s 55ms/step - loss: 0.3182 - accuracy: 0.9265 - val_loss: 0.2318 - val_accuracy: 0.9773
Epoch 31/40
65/65 [============================>.] - ETA: 0s - loss: 0.2990 - accuracy: 0.9408
Epoch 00031: val_loss did not improve from 0.21570
66/65 [==============================] - 4s 55ms/step - loss: 0.3002 - accuracy: 0.9402 - val_loss: 0.2568 - val_accuracy: 0.9640
Epoch 32/40
64/65 [============================>.] - ETA: 0s - loss: 0.2792 - accuracy: 0.9437
Epoch 00032: val_loss did not improve from 0.21570
66/65 [==============================] - 4s 54ms/step - loss: 0.2786 - accuracy: 0.9440 - val_loss: 0.2447 - val_accuracy: 0.9621
Epoch 33/40
65/65 [============================>.] - ETA: 0s - loss: 0.2810 - accuracy: 0.9403
Epoch 00033: val_loss did not improve from 0.21570
66/65 [===================

1. 列出路径下的文件名或图片名并且存入list列表，进行for循环取出，构建绝对路径即可对该文件进行遍历操作。

2. imread函数有两个参数，第一个参数是图片路径，第二个参数表示读取图片的形式，有三种：
   cv2.IMREAD_COLOR：加载彩色图片，这个是默认参数，可以直接写1。
   cv2.IMREAD_GRAYSCALE：以灰度模式加载图片，可以直接写0。
   cv2.IMREAD_UNCHANGED：包括alpha，可以直接写-1

3. p1是需要转换的图片，p2是转换成何种格式。
   cv2.COLOR_BGR2RGB 将BGR格式转换成RGB格式
   cv2.COLOR_BGR2GRAY 将BGR格式转换成灰度图片

4. os.path.sep:路径分隔符  linux下是’/’

5. 一、dump()方法
    pickle.dump(obj, file, [,protocol])
    注释：序列化对象，将对象obj保存到文件file中去。参数protocol是序列化模式，默认是0（ASCII协议，表示以文本的形式进行序列化），protocol的值   还可以是1和2（1和2表示以二进制的形式进行序列化。其中，1是老式的二进制协议；2是新二进制协议）。file表示保存到的类文件对象，file必须有       write()接口，file可以是一个以'w'打开的文件或者是一个StringIO对象，也可以是任何可以实现write()接口的对象。
   二、load()方法
    pickle.load(file)
   注释：反序列化对象，将文件中的数据解析为一个python对象。file中有read()接口和readline()接口

6. conv2d主要实现了输入张量与设定卷积核的卷积操作
    tf.nn.conv2d(
    input,  输入
    filter,  卷积核
    strides,  滑动步长
    padding,   图像边沿填充的方式
    use_cudnn_on_gpu=True,
    data_format='NHWC',
    dilations=[1, 1, 1, 1],
    name=None
)
7. - categorical_crossentropy要求target为onehot编码。
   - sparse_categorical_crossentropy要求target为非onehot编码，函数内部进行onehot编码实现。
